In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
! pip install datasets transformers rouge-score nltk

import transformers

model_checkpoint = "t5-base"

     |████████████████████████████████| 266kB 7.3MB/s 
     |████████████████████████████████| 2.5MB 12.9MB/s 
     |████████████████████████████████| 122kB 51.0MB/s 
     |████████████████████████████████| 245kB 44.8MB/s 
     |████████████████████████████████| 3.3MB 45.4MB/s 
     |████████████████████████████████| 901kB 35.9MB/s 
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset('csv', data_files= '/content/drive/MyDrive/articles1.csv')
metric = load_metric("rouge")

Using custom data configuration default-bf79a20a2dfac1d3


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bf79a20a2dfac1d3/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


In [ ]:
!pip install sentencepiece

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

     |████████████████████████████████| 1.2MB 7.6MB/s 


In [ ]:
if model_checkpoint in ["t5-base"]:
    prefix = "heading: "
else:
    prefix = ""

In [ ]:
max_input_length = 256
max_target_length = 64

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["content"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    "heading-generation",
    learning_rate=10e-5,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: publication, date, month, content, title, url, author, year, id, Unnamed: 0.
***** Running training *****
  Num examples = 50000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9375


Step,Training Loss
500,2.134400
1000,nan
1500,nan
2000,nan
2500,nan
3000,nan
3500,nan
4000,nan
4500,nan
5000,nan


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1299: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
Saving model checkpoint to heading-generation/checkpoint-500
Configuration saved in heading-generation/checkpoint-500/config.json
Model weights saved in heading-generation/checkpoint-500/pytorch_model.bin
tokenizer config file saved in heading-generation/checkpoint-500/tokenizer_config.json
Special tokens file saved in heading-generation/checkpoint-500/special_tokens_map.json
Copy vocab file to heading-generation/checkpoint-500/spiece.model
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1299: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continui

TrainOutput(global_step=9375, training_loss=nan, metrics={'train_runtime': 4837.4325, 'train_samples_per_second': 31.008, 'train_steps_per_second': 1.938, 'total_flos': 5.872728254445158e+16, 'train_loss': nan, 'epoch': 3.0})

In [ ]:
model.save_pretrained("/content/drive/My Drive/Heading")
tokenizer.save_pretrained("/content/drive/My Drive/Heading")

Configuration saved in /content/drive/My Drive/Heading/config.json
Model weights saved in /content/drive/My Drive/Heading/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/Heading/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/Heading/special_tokens_map.json
Copy vocab file to /content/drive/My Drive/Heading/spiece.model


('/content/drive/My Drive/Heading/tokenizer_config.json',
 '/content/drive/My Drive/Heading/special_tokens_map.json',
 '/content/drive/My Drive/Heading/spiece.model',
 '/content/drive/My Drive/Heading/added_tokens.json',
 '/content/drive/My Drive/Heading/tokenizer.json')